In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn

from pandas import Series, DataFrame
from pylab import rcParams
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report

In [ ]:
%matplotlib inline
rcParams['figure.figsize'] = 10, 8
sns.set_style('whitegrid')

A primeira coisa que vamos fazer é ler o conjunto de dados usando a função read_csv() do Pandas. 
Colocaremos esses dados em um DataFrame do Pandas, chamado "titanic", e nomearemos cada uma das colunas.

In [ ]:
url = 'https://raw.githubusercontent.com/BigDataGal/Python-for-Data-Science/master/titanic-train.csv'
titanic = pd.read_csv(url)
titanic.columns = ['PassengerId','Survived','Pclass','Name','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked']
titanic.head()

VARIABLE DESCRIPTIONS
Survived - Survival (0 = No; 1 = Yes);
Pclass - Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd);
Name - Name;
Sex - Sex;
Age - Age;
SibSp - Number of Siblings/Spouses Aboard;
Parch - Number of Parents/Children Aboard;
Ticket - Ticket Number;
Fare - Passenger Fare (British pound);
Cabin - Cabin;
Embarked - Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton);

Como estamos construindo um modelo para prever a sobrevivência de passageiros do Titanic, nosso alvo será a variável "Survived" do dataframe titanic.
Para ter certeza de que é uma variável binária, vamos usar a função countplot () do Seaborn.

In [ ]:
sns.countplot(x='Survived',data=titanic, palette='hls')

Ok, agora veja que a variavel Survived é binária

# Checking for missing values
É fácil checar missing values usando método isnull() com o método sum(), o número retornado condiz com a quantidade True para o teste, ou seja, quantidade de valores nulos nas variaveis

In [ ]:
titanic.isna().sum()

In [ ]:
titanic.info()

Ok, então existem 891 linhas no dataframe. Cabin é quase todo composto por missing values, então podemos eliminar essa variável completamente! Mas e quanto à idade? A age parece um preditor relevante para a sobrevivência, certo? Nós queremos manter as variáveis, mas tem 177 missing values. Precisamos encontrar uma maneira de nos preencher desses valores em falta!

# Lidando com missing values
Removendo missing values
Vamos além dos missing values... Vamos descartar todas as variáveis que não são relevantes para a predição de Survived.

In [ ]:
#E quanto ao nome de uma pessoa, número do bilhete e número de identificação do passageiro? 
#Eles são irrelavantes para prever a capacidade de sobrevivência. 
#E, como você se lembra, a variável cabine é quase todoa com missing values!!


titanic_data = titanic.drop(['PassengerId','Name','Ticket','Cabin'], 1)
titanic_data.head()

Agora, o dataframe foi reduzido para apenas variáveis relevantes, mas agora precisamos lidar com os valores ausentes na variável age.

# Imputing missing values
Vejamos como a idade do passageiro está relacionada à sua classe como passageiro no barco.

In [ ]:
dict_box = {}
for _, df in titanic_data.groupby(['Pclass','Sex']):
    Pclass = df.Pclass.values[0]
    Sex = df.Sex.values[0]
    Ages = df.Age.values
    dict_box.update({'{}_{}'.format(Pclass,Sex):Ages})

df_box = pd.DataFrame.from_dict(dict_box, orient='index').T
df_box.plot(kind='box')

Com esse diagrama podemos ver que existe alguma relação entre a idade do passageiro, a classe que ele ocupa e seu sexo. Por exemplo podemos afirmar que em média os passageiros da primeira classe são mais velhos que os das demais classes. Também podemos dizer que no geral a idade dos passageiros do sexo masculino é maior que os passageros do sexo feminino.

Podemos então criar uma regra para preencher as lacunas nas idades. A regra será: preencher as lacunas com a mediana de cada grupo do diagrama acima. Desta forma vamos preencher os valores faltantes de uma maneira um pouco mais acertiva.

Vamos dar uma olhada para saber que valores seriam estes exatamente.

In [ ]:
df_box.describe()

Vemos que para 1 classe temos para os valores medianos (50%) de idade:
    Mulher: 35 anos
    Homen: 40 anos
Para a classe 2:
    Mulher: 28 anos
    Homen: 30 anos
Para a classe 3:
    Mulher: 21.5 anos
    Homen: 25 anos

In [ ]:
titanic_data['Age'].fillna(titanic_data.groupby(['Pclass','Sex'])['Age'].transform('median'),inplace=True) 

Agora vamos preencher a variável embarked, que representa o porto de embarque.
Como há apenas dois valores faltantes podemos supor sem muito risco de introduzir ruído no dataset que esses valores faltantes correspondem ao porto com o maior número de embarques.

In [ ]:
titanic_data.groupby('Embarked')['Survived'].count()

O porto mais provável que alguém tenha embarcado é Southampton. Assim vamos preencher os valores faltantes com 'S'

In [ ]:
titanic_data['Embarked'].fillna('S',inplace=True)

Vamos verificar os valores nulos novamente

In [ ]:
titanic_data.isnull().sum()

Não temos mais valores nulos. Podemos prosseguir

# Converting categorical variables to a dummy indicators

In [ ]:
gender = pd.get_dummies(titanic_data['Sex'],drop_first=False)
gender.head()

In [ ]:
embark_location = pd.get_dummies(titanic_data['Embarked'],drop_first=False)
embark_location.head()

In [ ]:
#Ainda não alteramos os dados! Vamos alterar em seguida.
titanic_data.head()

In [ ]:
titanic_data.drop(['Sex', 'Embarked'],axis=1,inplace=True)
titanic_data.head()

In [ ]:
titanic_dmy = pd.concat([titanic_data,gender,embark_location],axis=1)
titanic_dmy.head()

Agora temos um conjunto de dados com todas as variáveis no formato correto!

# Validando independencia entre as variáveis

In [ ]:
sns.heatmap(titanic_dmy.corr())  

Fare e Pclass não sao independentes uma com a outra. A essa relação damos o nome de intercolinearidade, esta relação pode ser prejudicial para o nosso modelo de machine learning. Precisamos eliminar uma dessas features. Vamos olhar o diagrama de dispersão para obter mais evidências.

In [ ]:
titanic_dmy.plot(y='Fare', x='Pclass', kind='scatter')

Vamos retirar a coluna 'Pclass'

In [ ]:
titanic_dmy.drop(['Pclass'],axis=1,inplace=True)
titanic_dmy.head()

# Agora vamos lá!!
1º: Separar o conjunto em variavel resposta e variaveis de treinamento

In [ ]:
X = titanic_dmy.iloc[:,1:].values
y = titanic_dmy.iloc[:,0].values

Agora dividir em treino e teste (teste com 30%)

In [ ]:
#ToDo parte B
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criando o modelo

In [ ]:
from sklearn.linear_model import LogisticRegression



LogReg = LogisticRegression(random_state=42)
LogReg.fit(X_train, y_train)

# Testando o modelo

In [ ]:
y_pred = LogReg.predict(X_test)
y_pred[:10]

In [ ]:
y_test[:10]

# Avaliando o modelo
analisando o rotulo real com o rotulo estimado.

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred, normalize='true')


sns.heatmap(confusion_matrix, annot=True)

Vemos que acertamos 76% dos que sobreviveram porém 24% dos que sobreviveram foram estimados como não sobreviventes. Ou seja temos 24% de falso positivos.

Acertamos 86% dos que não sobreviveram porém 14% desses foram classificados erroneamente. Ou seja temos 14% de falso negativos.

### Precisão, Recall, F1, AUC

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
y_pred_proba = LogReg.predict_proba(X_test)[::,1]
auc = metrics.roc_auc_score(y_test, y_pred_proba)
auc

Acima vemos mais algumas métricas importantes para modelos de classificação. Recomendamos fortemente que estudem essas métricas de validação.

*Obs: Para um classificador ideal a métrica AUC (Area Under the Curve) é igual a 1. Essa métrica está diretamente relacionada a curva ROC

### Desafio 1: Calcular a acurácia

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred, normalize=True)

### Desafio 2: Plotar a curva ROC

In [ ]:
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
plt.plot(fpr,tpr,label="ROC curve, auc="+str(auc))
plt.legend(loc=4)
plt.show()